In [16]:
import pandas as pd
train=pd.read_csv("train.csv", index_col="_id")
train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target
_id,,,,,,,,,,,,,,,,,,,,,
df7489733b004bbe40d3d37b34f82419,54,technician,married,professional.course,unknown,no,no,cellular,aug,tue,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
905a0b9a5456ee962223033473666be3,36,services,single,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
02d69c07ec6d68e31d641edd45ce61cd,53,blue-collar,married,basic.4y,no,yes,no,cellular,aug,mon,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,0
e492fbe0cccaf67cdb5c0944dbc8a167,34,blue-collar,married,basic.9y,no,yes,yes,telephone,may,wed,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
ac2dc6f5835bdea8e71dd99b65208d07,29,services,single,university.degree,no,yes,no,telephone,aug,fri,...,1,999,0,nonexistent,-1.7,94.027,-38.3,0.890,4991.6,0


In [17]:
removeWithEmptyCol=["job","marital","education","default","housing","loan","contact"]

def removeEmptyDataForTraining(testData):    
    for col in removeWithEmptyCol:
        testData=testData[testData[col]!="unknown"]
    testData=testData[testData["duration"]!=0]

removeEmptyDataForTraining(train)    
    
target="target"
y=train[target]
train=train.drop([target], axis=1)  

In [18]:
from sklearn.cross_validation import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.10, random_state=50)
X_test=train
y_test=y


In [19]:
import numpy as np

#removingCol=["nr.employed", "day_of_week"]
removingCol=["day_of_week"]

def fillCatEmpty(data,colName,emptyName="unknown"):       
    empVal=data[data[colName]!=emptyName][colName].value_counts().index[0]
    data[colName]=data[colName].map(lambda val: empVal if val==emptyName else val)

catCol=["job","marital","education","default","housing","loan","contact"]

def numeralizeCat(data,colName,valdict):    
    data[colName]=data[colName].map(lambda val: valdict[val])    

def fillEmptyNum(data,colName,emptyNumVal):    
    mean=data[data[colName]!=emptyNumVal][colName].mean()        
    data[colName]=data[colName].map(lambda val: mean if val==emptyNumVal else val)

num_list=train.select_dtypes(include=[np.number]).columns.tolist()    

def addLogSqr(data):
    for col in num_list:
        data[col+"_2"]=data[col].map(lambda val:val**2)

def runData(testData):    
    testData=testData.drop(removingCol, axis=1)              
    
    for col in catCol:
        fillCatEmpty(testData,col)
    fillCatEmpty(testData,"poutcome","nonexistent")
    
    numeralizeCat(testData,"month",{'jan':1,'feb':2,'mar':3,'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8,'sep':9,'oct':10, 'nov':11,'dec':12})
    
    numeralizeCat(testData,"poutcome",{"failure":0,"success":1})
    
    testData = pd.get_dummies(testData, columns=catCol)     
    
    fillEmptyNum(testData,"duration",0)
    #fillEmptyNum(testData,"pdays",999)
    
    addLogSqr(testData)
    
    return testData

In [20]:
#X_train=runData(X_train)
X_test=runData(X_test)

In [21]:
def colAdded(target1,target2):
    res=[]
    for c in target1.columns:
        if c not in target2.columns:            
            res.append(c)
    for c in res:
        target2[c]=0

#colAdded(X_train,X_test)
#colAdded(X_test,X_train)

In [22]:
#X_train.shape, X_train.shape

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [24]:
from tqdm import tnrange
res=[]
raMax=0
for criterion in ["gini","entropy"]:
    for splitter in ["best","random"]:
        for max_depth in tnrange(1,100):            
            dt=DecisionTreeClassifier(criterion=criterion,splitter=splitter,max_depth=max_depth)
            #dt.fit(X_train,y_train)
            #ra=roc_auc_score(y_test,dt.predict_proba(X_test)[:,1])
            ra=cross_val_score(dt,X_test,y_test,scoring='roc_auc',cv=10).mean()

            if ra>raMax:
                raMax=ra
                res=[criterion,splitter,max_depth,ra]
print(res)            



['entropy', 'best', 6, 0.9353520758500695]


In [25]:
dtc=DecisionTreeClassifier()
#dtc.fit(X_train,y_train)
#ra=roc_auc_score(y_test,dtc.predict_proba(X_test)[:,1])
print(cross_val_score(dtc,X_test,y_test,scoring='roc_auc',cv=10).mean())
lr=LogisticRegression(penalty='l1')
#lr.fit(X_train,y_train)
#ra=roc_auc_score(y_test,lr.predict_proba(X_test)[:,1])
print(cross_val_score(lr,X_test,y_test,scoring='roc_auc',cv=10).mean())
kn=KNeighborsClassifier()
#kn.fit(X_train,y_train)
#ra=roc_auc_score(y_test,kn.predict_proba(X_test)[:,1])
print(cross_val_score(kn,X_test,y_test,scoring='roc_auc',cv=10).mean())


0.7262935657337447
0.9285728920819087
0.8666064232079063


In [26]:
from tqdm import tnrange
res=[]
raMax=0
for max_depth in tnrange(1,100):
    kn=KNeighborsClassifier(n_neighbors=max_depth)
    #kn.fit(X_train,y_train)            
    #ra=roc_auc_score(y_test,kn.predict_proba(X_test)[:,1])    
    ra=cross_val_score(kn,X_test,y_test,scoring='roc_auc',cv=10).mean()
    if ra>raMax:
        raMax=ra
        res=[max_depth,ra]        
print(res)  


[93, 0.9292704877793838]


In [105]:
realTest=pd.read_csv("test.csv", index_col="_id")
realTest=runData(realTest)

In [106]:
train=pd.read_csv("train.csv", index_col="_id")
removeEmptyDataForTraining(train) 
target="target"
y=train[target]
train=train.drop([target], axis=1) 
train=runData(train,True)

In [108]:
dt=DecisionTreeClassifier(criterion='entropy',splitter='best',max_depth=6)
#dt=KNeighborsClassifier(87)
dt.fit(train,y)
predict= dt.predict_proba(realTest)
result=pd.DataFrame({"_id":realTest.index, "target":predict[:,1]})
result.to_csv("result.csv",index=False)